In [ ]:
from pymongo import MongoClient
import os
from multiprocessing.pool import ThreadPool as Pool
import cv2
import numpy as np
import utils.const as const

In [ ]:
#### wrote this code for colab where "dir" is not in colab's memory but in G Drive

db=const.grpId
collection=const.PostCollection
dir=db
outPutFile='manually.csv'

dir2='targetImages'

pool_size = 20  # your "parallelness"

In [ ]:
client = MongoClient()
myCollect=client[db][collection]
print(len(os.listdir(dir)))

In [ ]:
addrList=[]

In [ ]:
for post in myCollect.find():
  if 'comments' in post:
    for comment in post['comments']:
      if 'attachment' in comment and 'images' in comment['attachment']:
        for k in comment['attachment']['images']:
          addr=os.path.join(dir,post["_id"],'comments',comment['_id'],'attachment','images',k)
          addrList.append(addr)

print(len(addrList))

In [ ]:
print(addrList[0])

In [ ]:
if not os.path.isdir(dir2):
  os.mkdir(dir2)
else:
  print(dir2,"exist")

In [ ]:
def getNewName(addr):
  arr=addr.split('/')
  return arr[6]+'_'+arr[8]+'_I'+arr[11]

print(getNewName(addrList[0]))

In [ ]:
def worker(l,r):
    for i in range(l,r):
      os.system('cp '+addrList[i]+' '+os.path.join(dir2,getNewName(addrList[i])))

In [ ]:
batchSize=(int)(len(addrList)/pool_size)
if pool_size*batchSize!=len(addrList):
  batchSize+=1

print(batchSize)

In [ ]:
pool = Pool(pool_size)

for i in range(pool_size):
    pool.apply_async(worker, (i*batchSize,min(len(addrList),(i+1)*batchSize)))

In [ ]:
pool.close()
pool.join()

In [ ]:
!du -hs targetImages

In [ ]:
images=[]
for i in range(len(addrList)):
  addrList[i]=getNewName(addrList[i])
  im=cv2.imread(os.path.join(dir2,addrList[i]))
  im=cv2.resize(im,(256,256))

  images.append(im)

In [ ]:
images=np.array(images)

In [ ]:
un=np.unique(images,axis=0)
print(len(un))

In [ ]:
dir3=dir2+'2'
if not os.path.isdir(dir3):
  os.mkdir(dir3)
else:
  print(dir3,"exist")

for i in range(len(un)):
  cv2.imwrite(os.path.join(dir3,str(i+1)+'.jpg'),un[i])

In [ ]:
!zip -r -9 manualImages.zip targetImages2

In [ ]:
similarImages=[]
for i in range(len(un)):
  similarImages.append(str(i+1)+',,')

for i in range(len(images)):
  k=-1;
  for j in range(len(un)):
    if (images[i]==un[j]).all():
      k=j
      break
  similarImages[k]+=addrList[i]+' '

In [ ]:
with open(outPutFile,'w') as fp:
  fp.write('image_id,your_sentimental_tags,image_ids(ignore)\n')

  for i in range(len(similarImages)):
    fp.write(similarImages[i]+'\n')
